In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data

import time, json, datetime
from tqdm import tqdm
import sys, getopt

import time
import numpy as np
import pandas as pd
from sklearn.metrics import log_loss, roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

device = torch.device('cuda:3') if torch.cuda.is_available() else torch.device('cpu')
nono=[]
# nono=['userid_duration_mean','userid_click_mean','docid_click_mean','refresh','keyword0_click_mean','refresh_duration_mean','refresh_click_count']
pp='3.0'

In [3]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [4]:
# from transformers import AutoTokenizer, AutoModel
# import torch.nn as nn
# import torch.nn.functional as F

# class fn_cls(nn.Module):
#     def __init__(self,device,nume_fea_size):
#         super(fn_cls, self).__init__()
#         self.model = AutoModel.from_pretrained("bert-base-chinese")
#         self.model.resize_token_embeddings(len(tokenizer))##############
#         self.model.to(device)
# #         self.dropout = nn.Dropout(0.5)
#         self.l1 = nn.Linear(768, 1)
#         self.l2 = nn.Linear(nume_fea_size, 1)
#         self.l3 = nn.Linear(2, 1)
#     def forward(self, nume_fea, input_ids, attention_mask):
#         outputs = self.model(input_ids, attention_mask=attention_mask)
# #         print(outputs[0])torch.Size([8, 100, 768])
# #         print(outputs[1])torch.Size([8, 768])
# #         print(outputs[0][:,0,:])torch.Size([8, 768])
#         bert_cls = outputs[1]
# #         x = self.dropout(x)
#         x_bert = self.l1(bert_cls)
    
#         x_dense= self.l2(nume_fea)
#         x=self.l3(torch.cat((x_bert,x_dense),1))
        
#         return x

In [13]:
from transformers import AutoTokenizer, AutoModel
import torch.nn as nn
import torch.nn.functional as F
sigmoid=nn.Sigmoid()

class fn_cls(nn.Module):
    def __init__(self,device,nume_fea_size):
        super(fn_cls, self).__init__()
#         self.dropout = nn.Dropout(0.5)
        self.l1 = nn.Linear(300, 1)
        self.l2 = nn.Linear(nume_fea_size, 1)
        self.l3 = nn.Linear(2, 1)
    def forward(self, nume_fea, emb):
        x_title = self.l1(emb)
        x_dense= self.l2(nume_fea)
        x=self.l3(torch.cat((x_title,x_dense),1))
        
        return sigmoid(x_title)

In [6]:
# def train_and_eval(model, train_loader, valid_loader, epochs, optimizer, loss_fcn, scheduler, device,best_auc,j,ii):
#     import time
#     for _ in range(epochs):
#         """训练部分"""
        
#         model.to(device)
#         model.train()
        
#         print("Current lr : {}".format(optimizer.state_dict()['param_groups'][0]['lr']))
#         write_log('Epoch: {}:{}'.format(j,ii))
#         train_loss_sum = 0.0
#         start_time = time.time()
#         for idx, x in enumerate(train_loader):
#             doc_id, nume_fea, label = x[0], x[1], x[2]
#             doc_id=pd.DataFrame(doc_id,columns=['docid'])
        
#             doc_title=pd.merge(doc_id,title_,how='left',on='docid')
#             input_ids=torch.LongTensor(np.stack(doc_title['input_ids'],axis=0))
#             attention_mask=torch.LongTensor(np.stack(doc_title['attention_mask'],axis=0))
            
            
#             nume_fea, input_ids, attention_mask, label = nume_fea.to(device), input_ids.to(device), attention_mask.to(device), label.float().to(device)
# #             print(input_ids)
#             pred = model(nume_fea, input_ids, attention_mask).view(-1)
#             loss = loss_fcn(pred, label)
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()

#             train_loss_sum += loss.cpu().item()
#             if (idx + 1) % 50 == 0 or (idx + 1) == len(train_loader):
#                 write_log("Epoch {:04d} | Step {:04d} / {} | Loss {:.4f} | Time {:.4f}".format(
#                     _ + 1, idx + 1, len(train_loader), train_loss_sum / (idx + 1), time.time() - start_time))
#         scheduler.step()
#         """推断部分"""
#         model.eval()
#         with torch.no_grad():
#             valid_labels, valid_preds = [], []
#             for idx, x in tqdm(enumerate(valid_loader)):
#                 doc_id, nume_fea, label = x[0], x[1], x[2]
#                 doc_id=pd.DataFrame(doc_id,columns=['docid'])
                
#                 doc_title=pd.merge(doc_id,title_,how='left',on='docid')
#                 input_ids=torch.LongTensor(np.stack(doc_title['input_ids'],axis=0))
#                 attention_mask=torch.LongTensor(np.stack(doc_title['attention_mask'],axis=0))
                
#                 nume_fea, input_ids, attention_mask = nume_fea.to(device), input_ids.to(device), attention_mask.to(device)
#                 pred = model(nume_fea, input_ids, attention_mask).reshape(-1).data.cpu().numpy().tolist()
#                 valid_preds.extend(pred)
#                 valid_labels.extend(label.cpu().numpy().tolist())
#         cur_auc = roc_auc_score(valid_labels, valid_preds)
#         if cur_auc > best_auc:
#             best_auc = cur_auc
#             import time
#             end=time.time()
#             torch.save(model.state_dict(), "../../data/wj/bert_best/"+pp+"/bert_best_"+str(round(best_auc,4))+"_"+str(j)+"_"+str(ii)+"_"+str(time.strftime('%m_%d_%H_%M_%S'))+".pth")
#         write_log('Current AUC: %.6f, Best AUC: %.6f\n' % (cur_auc, best_auc))
#     return best_auc

def train_and_eval(model, train_loader, valid_loader, epochs, optimizer, loss_fcn, scheduler, device,best_auc,j,ii):
    import time
    for _ in range(epochs):
        """训练部分"""
        
        model.to(device)
        model.train()
        
        print("Current lr : {}".format(optimizer.state_dict()['param_groups'][0]['lr']))
        write_log('Epoch: {}:{}'.format(j,ii))
        train_loss_sum = 0.0
        start_time = time.time()
        for idx, x in enumerate(train_loader):
            doc_id, nume_fea, label = x[0], x[1], x[2]
            doc_id=pd.DataFrame(doc_id,columns=['docid'])
            doc_title=pd.merge(doc_id,title_,how='left',on='docid')
            title_0=torch.FloatTensor(np.stack(doc_title['title_'],axis=0))
            
            nume_fea, title_0, label = nume_fea.to(device), title_0.to(device), label.float().to(device)
#             print(input_ids)
            pred = model(nume_fea, title_0).view(-1)
            loss = loss_fcn(pred, label)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss_sum += loss.cpu().item()
            if (idx + 1) % 50 == 0 or (idx + 1) == len(train_loader):
                write_log("Epoch {:04d} | Step {:04d} / {} | Loss {:.4f} | Time {:.4f}".format(
                    _ + 1, idx + 1, len(train_loader), train_loss_sum / (idx + 1), time.time() - start_time))
        scheduler.step()
        """推断部分"""
        model.eval()
        with torch.no_grad():
            valid_labels, valid_preds = [], []
            for idx, x in tqdm(enumerate(valid_loader)):
                doc_id, nume_fea, label = x[0], x[1], x[2]
                doc_id=pd.DataFrame(doc_id,columns=['docid'])
                doc_title=pd.merge(doc_id,title_,how='left',on='docid')
                title_0=torch.FloatTensor(np.stack(doc_title['title_'],axis=0))
                
                nume_fea, title_0 = nume_fea.to(device), title_0.to(device)
                pred = model(nume_fea, title_0).reshape(-1).data.cpu().numpy().tolist()
                valid_preds.extend(pred)
                valid_labels.extend(label.cpu().numpy().tolist())
        cur_auc = roc_auc_score(valid_labels, valid_preds)
        if cur_auc > best_auc:
            best_auc = cur_auc
            import time
            end=time.time()
            torch.save(model.state_dict(), "../../data/wj/bert_best/"+pp+"/bert_best_"+str(round(best_auc,4))+"_"+str(j)+"_"+str(ii)+"_"+str(time.strftime('%m_%d_%H_%M_%S'))+".pth")
        write_log('Current AUC: %.6f, Best AUC: %.6f\n' % (cur_auc, best_auc))
    return best_auc


import pytz
import datetime
tz = pytz.timezone('Asia/Shanghai')

# 定义日志（data文件夹下，同级目录新建一个data文件夹）
def write_log(w):
    file_name = '../../data/wj/bert_best/'+pp+'/' + datetime.date.today().strftime('%m%d') + "_{}.log".format("bert")
    t0 = datetime.datetime.now(tz).strftime('%H:%M:%S')
    info = "{} : {}".format(t0, w)
    print(info)
    with open(file_name, 'a') as f:
        f.write(info + '\n')
        

In [7]:
doc_feat = pd.read_pickle('../../data/wj/doc.pkl')
user_feat = pd.read_pickle('../../data/wj/user.pkl')

sparse_features = ['userid', 'docid', 'network', 'hour', 'device', 'os', 'province',
                   'city', 'age', 'gender', 'category1st', 'category2nd',
                   'pub_date', 'keyword0', 'keyword1', 'keyword2', 'keyword3', 'keyword4']

dense_features0 = ['refresh', 'picnum',
                  'userid_click_mean','userid_click_count' ,'userid_duration_mean' ,'userid_picnum_mean',
                   'docid_click_mean','docid_click_count','docid_duration_mean','docid_picnum_mean',
                    'category1st_click_mean','category1st_click_count','category1st_duration_mean','category1st_picnum_mean',
                    'category2nd_click_mean','category2nd_click_count','category2nd_duration_mean','category2nd_picnum_mean',
                    'keyword0_click_mean','keyword0_click_count','keyword0_duration_mean','keyword0_picnum_mean',
                 'network_click_mean', 'network_click_count', 'network_duration_mean', 
                  'refresh_click_mean', 'refresh_click_count', 'refresh_duration_mean',
                  'device_click_mean', 'device_click_count', 'device_duration_mean', 
                  'os_click_mean', 'os_click_count', 'os_duration_mean', 
                  'province_click_mean', 'province_click_count', 'province_duration_mean', 
                  'city_click_mean', 'city_click_count', 'city_duration_mean', 
                  'age_click_mean', 'age_click_count', 'age_duration_mean', 
                  'gender_click_mean', 'gender_click_count', 'gender_duration_mean'
                 ]

dense_features=[]
for i in dense_features0:
    if i not in nono:
        dense_features.append(i)
print(len(dense_features0),len(dense_features))

cate_fea_nuniqs = []
cate_fea_nuniqs.append(user_feat['userid'].nunique() + 1)
cate_fea_nuniqs.append(doc_feat['docid'].nunique() + 1)
cate_fea_nuniqs.append(6)  # network
cate_fea_nuniqs.append(13)  # hour
cate_fea_nuniqs.append(user_feat['device'].nunique())
cate_fea_nuniqs.append(user_feat['os'].nunique())
cate_fea_nuniqs.append(user_feat['province'].nunique())
cate_fea_nuniqs.append(user_feat['city'].nunique())
cate_fea_nuniqs.append(user_feat['age'].nunique())
cate_fea_nuniqs.append(user_feat['gender'].nunique())
cate_fea_nuniqs.append(doc_feat['category1st'].nunique())
cate_fea_nuniqs.append(doc_feat['category2nd'].nunique())
cate_fea_nuniqs.append(doc_feat['pub_date'].nunique())
keyword_nunique = max(doc_feat['keyword0'].max(), doc_feat['keyword1'].max(), doc_feat['keyword2'].max()
                      , doc_feat['keyword3'].max(), doc_feat['keyword4'].max()) + 1
cate_fea_nuniqs.append(keyword_nunique)
cate_fea_nuniqs.append(keyword_nunique)
cate_fea_nuniqs.append(keyword_nunique)
cate_fea_nuniqs.append(keyword_nunique)
cate_fea_nuniqs.append(keyword_nunique)

46 46


In [8]:
import pandas as pd
title_=pd.read_pickle('../../data/wj/title_embedding300.pkl')
del title_['title']
# title_

In [9]:
# import pandas as pd
# title_=pd.read_csv('../../data/wj/doc_title.csv')
# text=title_['title'].tolist()
# for i in range(len(text)):
#     if not isinstance(text[i], str):
#         print(text[i])
#         text[i]=''



# from transformers import AutoTokenizer, AutoModel
# # added_token=['##char##']
# # tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese",additional_special_tokens=added_token)
# tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")
# text2id=tokenizer(text, max_length=100, padding='max_length', truncation=True, return_tensors="pt")
# nput_ids=text2id["input_ids"].tolist()
# attention_mask=text2id["attention_mask"].tolist()
# title_['input_ids']=nput_ids
# title_['attention_mask']=attention_mask

# title_.head()

In [14]:
model = fn_cls(device,nume_fea_size=len(dense_features))

loss_fcn = nn.MSELoss()
loss_fcn = loss_fcn.to(device)
# optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)
optimizer = optim.Adam(model.parameters(), lr=0.005, weight_decay=0.001)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.8)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.8)

In [11]:
# import pandas as pd


# len_=81671133
# best_auc=0
# for j in range(5):
#     ii=-1
#     for i in range(0,len_,round(len_/10)):
#         ii+=1
#         right=i+round(len_/10)
#         if right>=len_:
#             break
#         print(i,right)
#         df_train_user_doc=pd.read_pickle('../../data/wj/df_train_user_doc_0_1_'+str(i)+'_'+str(right)+'_64.pkl')
# #         df_train_user_doc=pd.merge(df_train_user_doc,title_,how='left',on='docid')
        
#         train, valid = train_test_split(df_train_user_doc, test_size=0.25, random_state=2021)
# #         train_loader，valid_loader
#         train_dataset = Data.TensorDataset(torch.LongTensor(np.stack(train['docid'].values,axis=0)),
#                                            torch.FloatTensor(train[dense_features].values),
#                                            torch.FloatTensor(train['click'].values), )
#         train_loader = Data.DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
        
#         valid_dataset = Data.TensorDataset(torch.LongTensor(np.stack(valid['docid'].values,axis=0)),
#                                            torch.FloatTensor(valid[dense_features].values),
#                                            torch.FloatTensor(valid['click'].values))
#         valid_loader = Data.DataLoader(dataset=valid_dataset, batch_size=16, shuffle=False)
# #         train
#         print('train')
#         epoch_=1
#         best_auc=train_and_eval(model, train_loader, valid_loader, epoch_, optimizer, loss_fcn, scheduler, device,best_auc,j,ii)

In [ ]:
import pandas as pd


len_=81671133
best_auc=0
for j in range(5):
    ii=-1
    for i in range(0,len_,round(len_/10)):
        ii+=1
        right=i+round(len_/10)
        if right>=len_:
            break
        print(i,right)
        df_train_user_doc=pd.read_pickle('../../data/wj/df_train_user_doc_0_1_'+str(i)+'_'+str(right)+'_64.pkl')
#         df_train_user_doc=pd.merge(df_train_user_doc,title_,how='left',on='docid')
        
        train, valid = train_test_split(df_train_user_doc, test_size=0.25, random_state=2021)
#         train_loader，valid_loader
        train_dataset = Data.TensorDataset(torch.LongTensor(np.stack(train['docid'].values,axis=0)),
                                           torch.FloatTensor(train[dense_features].values),
                                           torch.FloatTensor(train['click'].values), )
        train_loader = Data.DataLoader(dataset=train_dataset, batch_size=8192, shuffle=True)
        
        valid_dataset = Data.TensorDataset(torch.LongTensor(np.stack(valid['docid'].values,axis=0)),
                                           torch.FloatTensor(valid[dense_features].values),
                                           torch.FloatTensor(valid['click'].values))
        valid_loader = Data.DataLoader(dataset=valid_dataset, batch_size=8192, shuffle=False)
#         train
        print('train')
        epoch_=1
        best_auc=train_and_eval(model, train_loader, valid_loader, epoch_, optimizer, loss_fcn, scheduler, device,best_auc,j,ii)

0 8167113
train
Current lr : 0.005
00:26:17 : Epoch: 0:0
00:26:34 : Epoch 0001 | Step 0050 / 748 | Loss 0.1350 | Time 16.9269
00:26:49 : Epoch 0001 | Step 0100 / 748 | Loss 0.1298 | Time 32.3504
00:27:05 : Epoch 0001 | Step 0150 / 748 | Loss 0.1278 | Time 47.8488
00:27:21 : Epoch 0001 | Step 0200 / 748 | Loss 0.1270 | Time 63.6701
00:27:37 : Epoch 0001 | Step 0250 / 748 | Loss 0.1264 | Time 79.8634
00:27:52 : Epoch 0001 | Step 0300 / 748 | Loss 0.1260 | Time 95.3600
00:28:08 : Epoch 0001 | Step 0350 / 748 | Loss 0.1257 | Time 110.6146
00:28:23 : Epoch 0001 | Step 0400 / 748 | Loss 0.1255 | Time 125.5845
00:28:38 : Epoch 0001 | Step 0450 / 748 | Loss 0.1253 | Time 141.0486
00:28:54 : Epoch 0001 | Step 0500 / 748 | Loss 0.1251 | Time 156.6900
00:29:09 : Epoch 0001 | Step 0550 / 748 | Loss 0.1250 | Time 172.0465
00:29:25 : Epoch 0001 | Step 0600 / 748 | Loss 0.1250 | Time 187.7130
00:29:40 : Epoch 0001 | Step 0650 / 748 | Loss 0.1249 | Time 202.8594
00:29:56 : Epoch 0001 | Step 0700 / 748

250it [01:13,  3.41it/s]


00:31:25 : Current AUC: 0.533739, Best AUC: 0.533739

8167113 16334226
train
Current lr : 0.004
00:31:55 : Epoch: 0:1
00:32:12 : Epoch 0001 | Step 0050 / 748 | Loss 0.1228 | Time 16.4495
00:32:27 : Epoch 0001 | Step 0100 / 748 | Loss 0.1234 | Time 31.6977
00:32:42 : Epoch 0001 | Step 0150 / 748 | Loss 0.1236 | Time 47.1438
00:32:58 : Epoch 0001 | Step 0200 / 748 | Loss 0.1236 | Time 62.5096
00:33:13 : Epoch 0001 | Step 0250 / 748 | Loss 0.1235 | Time 78.1752
00:33:29 : Epoch 0001 | Step 0300 / 748 | Loss 0.1235 | Time 93.6950
00:33:46 : Epoch 0001 | Step 0350 / 748 | Loss 0.1236 | Time 110.4730
00:34:03 : Epoch 0001 | Step 0400 / 748 | Loss 0.1237 | Time 127.4348
00:34:20 : Epoch 0001 | Step 0450 / 748 | Loss 0.1236 | Time 144.3466
00:34:36 : Epoch 0001 | Step 0500 / 748 | Loss 0.1237 | Time 160.5123
00:34:51 : Epoch 0001 | Step 0550 / 748 | Loss 0.1237 | Time 175.8768
00:35:06 : Epoch 0001 | Step 0600 / 748 | Loss 0.1237 | Time 191.1427
00:35:22 : Epoch 0001 | Step 0650 / 748 | Loss 0

250it [01:15,  3.33it/s]


00:37:09 : Current AUC: 0.536324, Best AUC: 0.536324

16334226 24501339
train
Current lr : 0.0032
00:37:40 : Epoch: 0:2
00:37:56 : Epoch 0001 | Step 0050 / 748 | Loss 0.1236 | Time 16.1882
00:38:12 : Epoch 0001 | Step 0100 / 748 | Loss 0.1239 | Time 32.2491
00:38:28 : Epoch 0001 | Step 0150 / 748 | Loss 0.1237 | Time 48.6090
00:38:44 : Epoch 0001 | Step 0200 / 748 | Loss 0.1239 | Time 64.0721
00:38:59 : Epoch 0001 | Step 0250 / 748 | Loss 0.1238 | Time 79.6333
00:39:17 : Epoch 0001 | Step 0300 / 748 | Loss 0.1238 | Time 96.8439
00:39:33 : Epoch 0001 | Step 0350 / 748 | Loss 0.1239 | Time 113.2427
00:39:49 : Epoch 0001 | Step 0400 / 748 | Loss 0.1239 | Time 129.6099
00:40:06 : Epoch 0001 | Step 0450 / 748 | Loss 0.1238 | Time 145.7006
00:40:21 : Epoch 0001 | Step 0500 / 748 | Loss 0.1238 | Time 161.2362
00:40:37 : Epoch 0001 | Step 0550 / 748 | Loss 0.1237 | Time 176.9408
00:40:53 : Epoch 0001 | Step 0600 / 748 | Loss 0.1237 | Time 193.0584
00:41:08 : Epoch 0001 | Step 0650 / 748 | Loss

63it [00:18,  3.52it/s]

In [22]:
df_test_user_doc=pd.read_pickle('../../data/wj/df_test_user_doc_64_new.pkl')

In [25]:
import time
import pandas as pd
def predict(test_df, s_feat, den_feat, model, device,modeln):
    test_dataset = Data.TensorDataset(torch.LongTensor(np.stack(test_df['docid'].values,axis=0)),
                                           torch.FloatTensor(test_df[dense_features].values))
    test_loader = Data.DataLoader(dataset=test_dataset, batch_size=4096, shuffle=False)
    
    
    with torch.no_grad():
        test_preds = []
        for idx, x in tqdm(enumerate(test_loader)):
            doc_id, nume_fea = x[0], x[1]
            doc_id=pd.DataFrame(doc_id,columns=['docid'])
            doc_title=pd.merge(doc_id,title_,how='left',on='docid')
            title_0=torch.FloatTensor(np.stack(doc_title['title_'],axis=0))

            nume_fea, title_0 = nume_fea.to(device), title_0.to(device)
            pred = model(nume_fea, title_0).reshape(-1).data.cpu().numpy().tolist()
            
            test_preds.extend(pred)
        id_list = list(range(0, len(test_preds)))
        out_dict = {"id": id_list, "pred": test_preds}
        out_df = pd.DataFrame(out_dict)
        out_df.to_csv('../../data/wj/bert_best/'+pp+'/bert_result_user_'+modeln+'.csv', sep=',', header=False, index=False)

In [26]:
modeln='0.7598_0_8_12_21_13_24_11'
model = fn_cls(device,nume_fea_size=len(dense_features))
model.load_state_dict(torch.load('../../data/wj/bert_best/'+pp+'/bert_best_'+modeln+'.pth'))

model = model.to(device)
model.eval()  # 把模型转为test模式
predict(df_test_user_doc, sparse_features, dense_features, model, device,modeln)


13it [00:02,  6.17it/s]
